In [1]:
# Install required packages if not already installed
import sys
import subprocess
def install(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
for pkg in ['xarray', 'rasterio', 'pandas', 'netCDF4']:
    try:
        __import__(pkg)
    except ImportError:
        install(pkg)

import os
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import pandas as pd

# Path to NetCDF file and output directory
nc_path = '/Users/silvanragettli/Library/CloudStorage/OneDrive-hydrosolutionsltd(2)/Shared_with_Justine/13_Precipitation/Precipitation_RhiresD/RhiresD_2001_2024/RhiresD_ch01h.swiss.lv95_202401010000_202412310000.nc'
out_dir = '/Users/silvanragettli/Library/CloudStorage/OneDrive-hydrosolutionsltd(2)/Shared_with_Justine/13_Precipitation/Precipitation_RhiresD/tif_2024'
os.makedirs(out_dir, exist_ok=True)


  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 89.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 99.7 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [xarray]2m4/5 [xarray]
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1.2-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 103.5 MB/s  0:00:00
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
Usi

In [8]:
year = 2023

In [ ]:
# Open NetCDF file
ds = xr.open_dataset(nc_path)

# Assume variable name is 'RhiresD' (change if needed)
var_name = 'RhiresD' if 'RhiresD' in ds.variables else list(ds.data_vars)[0]

metadata = []

for i, t in enumerate(ds['time']):
    date = pd.to_datetime(str(t.values))
    if date.year != year:
        continue  # Skip if not the selected year
    if date.month < 4 or date.month > 9:
        continue  # Skip months outside April-September
    arr = ds[var_name].isel(time=i).values
    arr = arr[::-1, :]  # Flip vertically if needed
    tif_name = f"RhiresD_{date.strftime('%Y%m%d%H%M')}.tif"
    tif_path = os.path.join(out_dir, tif_name)

    # Get geotransform info (adjust if needed)
    lon = ds['E'].values
    lat = ds['N'].values
    pixel_width = abs(lon[1] - lon[0])
    pixel_height = abs(lat[1] - lat[0])
    transform = from_origin(lon.min(), lat.max(), pixel_width, pixel_height)
    # Write GeoTIFF
    with rasterio.open(
        tif_path, 'w',
        driver='GTiff',
        height=arr.shape[0],
        width=arr.shape[1],
        count=1,
        dtype=arr.dtype,
        crs='EPSG:2056',  # LV95 Swiss projection
        transform=transform
    ) as dst:
        dst.write(arr, 1)

    # Collect metadata
    metadata.append({
        'id_no': tif_name.replace('.tif', ''),
        'month': date.month,
        'year': date.year,
        'day': date.day,
        'SPACECRAFT_ID': 'RhiresD'
    })

# Save metadata CSV
meta_df = pd.DataFrame(metadata)
meta_df.to_csv(os.path.join(out_dir, f'metadata_{year}_RhiresD.csv'), index=False)

print('Extraction and metadata creation complete.')

Extraction and metadata creation complete.
